## Load latest headlines

In [1]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-07-22-31-19 +0000,nyt,"Immigration Agents Conduct More Raids in LA, C...",https://www.nytimes.com/2025/06/07/us/la-immig...
1,2025-06-07-22-15-19 +0000,nyt,Trump Warns of Consequences for Musk if He Bac...,https://www.nytimes.com/2025/06/07/us/politics...
2,2025-06-07-22-00-28 +0000,nyt,WorldPride Parade Attendees Celebrate in D.C. ...,https://www.nytimes.com/2025/06/07/us/worldpri...
3,2025-06-07-21-53-39 +0000,nyt,Trump Targets Workplaces as Immigration Crackd...,https://www.nytimes.com/2025/06/07/us/trump-ta...
4,2025-06-07-21-06-45 +0000,nyt,"The world is celebrating Pride in Washington, ...",https://www.nytimes.com/2025/06/07/us/the-worl...


## Calculate word frequencies

In [2]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_27196\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,15
10,musk,4
111,dies,4
90,ban,4
155,russian,4


## Rank headlines by score

In [3]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-06-07-09-00-00 +0000,wsj,Elon Musk and House Republicans both promised ...,https://www.wsj.com/politics/policy/republican...,37
1,2025-06-07-22-15-19 +0000,nyt,Trump Warns of Consequences for Musk if He Bac...,https://www.nytimes.com/2025/06/07/us/politics...,32
26,2025-06-07-10-00-07 +0000,wapo,Trump travel ban targets nations mired in civi...,https://www.washingtonpost.com/world/2025/06/0...,30
13,2025-06-07-18-04-22 +0000,nyt,Haitians Reel as Trump’s Travel Ban Tears Fami...,https://www.nytimes.com/2025/06/07/us/haiti-tr...,28
10,2025-06-07-18-34-42 +0000,nyt,"After His Trump Blowup, Musk May Be Out. But D...",https://www.nytimes.com/2025/06/07/us/politics...,28


## Select top headlines

In [4]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,37,2025-06-07-09-00-00 +0000,wsj,Elon Musk and House Republicans both promised ...,https://www.wsj.com/politics/policy/republican...
26,30,2025-06-07-10-00-07 +0000,wapo,Trump travel ban targets nations mired in civi...,https://www.washingtonpost.com/world/2025/06/0...
20,16,2025-06-07-14-39-12 +0000,nyt,Kilmar Abrego Garcia Returned to U.S. From El ...,https://www.nytimes.com/2025/06/06/us/politics...
23,16,2025-06-07-11-45-25 +0000,nyt,"Russian Spies Are Suspicious of China, Even as...",https://www.nytimes.com/2025/06/07/world/europ...
18,15,2025-06-07-15-10-19 +0000,nyt,Agents Use Military-Style Force Against Protes...,https://www.nytimes.com/2025/06/06/us/los-ange...
46,14,2025-06-07-02-21-40 +0000,wapo,Boston Consulting Group CEO apologizes for Isr...,https://www.washingtonpost.com/national-securi...
22,12,2025-06-07-12-18-44 +0000,nyt,D.C. Police Officer Sentenced to Prison for Le...,https://www.nytimes.com/2025/06/07/us/dc-polic...
21,11,2025-06-07-14-20-18 +0000,nyt,"Paul Durcan, Irish Poet of Tortured and Tender...",https://www.nytimes.com/2025/06/06/books/paul-...
14,10,2025-06-07-17-49-00 +0000,nyt,Trump’s Proposed Cut Would Deal Serious Setbac...,https://www.nytimes.com/2025/06/07/us/high-spe...
17,10,2025-06-07-17-06-17 +0000,nyt,Israel Says It Made Record Weapons Sales Abroa...,https://www.nytimes.com/2025/06/04/world/middl...


## Next steps